## LangChain 사용해서 지원사업 공고 파일(pdf, hwp, png)을 요약해보자!

In [1]:
from openai import OpenAI
from getpass import getpass
from tqdm import tqdm

from utils.get_text import get_hwp_text, get_pdf_text, get_image_text # pdf, hwp에서 텍스트를 추출하는 사용자 지정 함수 

- LangChain 모델이 참고할 비슷한 용어 사전
- 개발팀이 직접 정의
- (추후에 유사한 단어가 나오면 사전에 추가하는 기능도 구현할 예정)

In [2]:
# 정의한 비슷한 용어 사전
keyword_dictionary = {
                        '신청기간' : ['신청 기간', '접수 기간'],
                        '신청방법' : ['지원 대상', '사업 대상', '융자 대상'],
                        '지원제외' : ['지원 제외','지원 취소', '제외 대상', '제한 대상'],
                        '유의사항' : ['의무사항', '지원 취소 및 제외', '기타 안내사항'],
                        '제출 서류' : ['필요 서류', '서류 제출 목록'],
                        '선정 절차' : ['선정 절차', '평가 방법', '평가 방향', '우대 사항', '평가 방향', '선정 기준'],
                        '지원 내용' : ['지원 내용', '규모', '분야 및 대상', '범위 및 형태', '세부지원내용'],
                        '문의처' : ['문의 전화', '연락처', '지원 문의처', '문의 및 상담', '접수 및 문의'],
                        '기타' : ['기타']
                        }

#### model 구현 준비
- OpenAI 객체 생성
- 파일 텍스트 추출
- model이 참고할 기본 템플릿 작성

In [3]:
# 개인의 API key를 입력
MY_API_KEY = getpass("OpenAI API Key")

#OpenAI 객체 생성
client = OpenAI(api_key=MY_API_KEY)

In [5]:
# 파일 경로 설정 (백슬래시 문제 해결을 위해 r"" 사용)
file_path = f"금융\[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고.png"

# 지원사업명 저장 (지원경로를 분할하여 사용)
title = file_path.split('\\')[1].split('.')[0]

# 파일 타입 저장 (지원경로를 분할하여 사용)
file_type = file_path.split('\\')[1].split('.')[1]

# 지원사업 텍스트 추출 (PDF/HWP 처리 함수 필요)
if file_path.endswith(".pdf"):
    full_text = get_pdf_text(file_path)
elif file_path.endswith(".hwp"):
    full_text = get_hwp_text(file_path)
elif file_path.endswith(".png"):
    full_text = get_image_text(file_path)

print(full_text)

<>:2: SyntaxWarning: invalid escape sequence '\['
<>:2: SyntaxWarning: invalid escape sequence '\['
C:\Users\every\AppData\Local\Temp\ipykernel_15924\3066476994.py:2: SyntaxWarning: invalid escape sequence '\['
  file_path = f"금융\[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고.png"


당신의 부 2025 북구 소상공인 대출 지금 바로 신청하세요! 신청기간 (공통) 25. 3.4.~ 자금 소진 시 북구 디딤돌 3 특례보증 지원 - 무담보, 무이자, 무보증료 → 1년간 대출이자(5.5% 이내) 및 보증료 전액 지원(사후 입금) 지원대상 북구 관내 임차 소상공인 지원제외 : 자가 또는 무상임차인 경우, 2024년 특례보증 대출 이용자, 보증제외업종 대출조건 - 대출한도: 업체당 최대 2천만원 (관광사업자, 2020년 이후 재창업자 3천만원 우대) - 대출 및 상환 : 일시상환(1년) / 원금균등분할상환(5년) - 대출금리 : 금융기관별 상이 (취급 시점 산출금리 5.5% 이내) 취급은행 광주은행, 광주문화신협, 북구 새마을금고(11개), 광주농협 전 지점 *북구 새마을금고(11개) 1 : 광주북부, 광주중앙, 동부, 동운, 서방, 북광주, 빛고을, 예향, 정다운, 중흥, 태봉 문의 취급은행 영업점 또는 광주신용보증재단 홈페이지(www.gjsirbo.or.kr) 저신용·저소득 소상공인 포용금융 이차보전 지원 지원대상 북구 관내 저신용·저소득 소상공인 지원내용 광주은행 서민금융형 대출상품의 이자 5.0% 지원(1년간) * 서민금융형 대출상품 : 햇살론1 511, 햇살론뱅크, 최저신용자특례보증, 포용금융특별대출, KJB새희망홀씨Ⅱ 대출조건 최대 3,500만원 / 최장 10년 대출상품별 상이 2 취급은행 광주은행 전 지점 문의 광주은행 영업점, 서민금융진흥원 북광주통합지원센터(1397) 광주광역시북구


- 템플릿 생성

In [8]:
base_template = {f'''
📌 **[{title}] 요약 안내**

📅 **신청기간**  

📝 **신청방법**  

🚫 **지원제외 대상**   

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  

📞 **문의처**  

📢 **기타 안내**  
'''
}

#### LangChain 모델로 간단하게 요약해보기
- (고도화 예정)

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

template = """
당신은 파일 요약 전문가입니다. 사용자가 요구하는 파일을 제공하는 템플릿 양식에 맞게 요약해야합니다.
파일에 없는 내용은 공백으로 남겨두십시오.
요구하는 파일은 양식이 일정하지 않습니다. 정의된 사전을 선언해두었으니, 사전에 해당하는 단어가 나오면, 참고하여 요약해주십시오.

제공하는 템플릿 양식 : {base_template}
단어 정의 사전 : {keyword_dictionary}

---------------------------------------------------
사용자 요구 파일 : {text}

"""

# 파일 요약을 위한 프롬프트 템플릿을 정의
prompt = PromptTemplate.from_template(template)

# OpenAI의 GPT-4o 모델을 사용하여 채팅 모델을 생성
model = ChatOpenAI(
    model='gpt-4o',  # 사용할 언어 모델 지정
    api_key=MY_API_KEY  # OpenAI API 키 입력
)

# 모델의 출력을 문자열로 변환하는 출력 파서
output_parser = StrOutputParser()

# 체인(Chain) 구성: 프롬프트 → 모델 → 출력 파서 순서로 연결
chain = prompt | model | output_parser

In [13]:
print(chain.invoke({'base_template':base_template,
              'keyword_dictionary':keyword_dictionary,
              'text':full_text}))

```
📌 **[[광주] 북구 2025년 소상공인 대출 지원사업(특례보증ㆍ포용금융) 공고] 요약 안내**

📅 **신청기간**  
- 2025년 3월 4일부터 자금 소진 시까지

📝 **신청방법**  
- 북구 관내 임차 소상공인
- 북구 관내 저신용·저소득 소상공인

🚫 **지원제외 대상**  
- 자가 또는 무상임차인
- 2024년 특례보증 대출 이용자
- 보증제외업종

⚠ **유의사항**  

📑 **제출서류**  

📌 **선정절차**  

💰 **지원내용**  
- 무담보, 무이자, 무보증료 대출
- 1년간 대출이자(5.5% 이내) 및 보증료 전액 지원
- 광주은행 서민금융형 대출상품 이자 5.0% 지원(1년 간)
- 대출한도: 업체당 최대 2천만원(관광사업자, 2020년 이후 재창업자 3천만원 우대)
- 대출조건: 최대 3,500만원 / 최장 10년 (대출상품별 상이)

📞 **문의처**  
- 취급은행 영업점 또는 광주신용보증재단 홈페이지(www.gjsirbo.or.kr)
- 광주은행 영업점, 서민금융진흥원 북광주통합지원센터(1397)

📢 **기타 안내**  
- 취급은행: 광주은행, 광주문화신협, 북구 새마을금고(11개), 광주농협 전 지점
```
